In [1]:
# recreate the lebanoff et al 2018 evaluation setup
# adapted from here: 
# https://github.com/ucfnlp/multidoc_summarization/blob/ae30c9ee039d4ad5ff64fd2245faafc5a62c4dd7/decode.py

# installing pyrouge
# https://stackoverflow.com/questions/45894212/installing-pyrouge-gets-error-in-ubuntu

In [6]:
from pathlib import Path
import json
import os
import logging

from transformer_decoding.evaluate import lebanoff_2018_rouge, evaluate_rouge, print_mean

In [21]:
# TEMP_EVAL_DIR = Path('rouge_evaluation_tempdir')
# os.makedirs(TEMP_EVAL_DIR, exist_ok=True)


# DUC 2004
# evaluation_dataset = '/home/chris/projects/aylien/dynamic-ensembles/data/DUC2004/DUC2004_test.jsonl'

# system_hypotheses = '/home/chris/projects/aylien/dynamic-ensembles/data/DUC2004/system_hypotheses/2020-05-14/eval_predicted_summaries.out.1_doc_in_cluster'
# system_hypotheses = '/home/chris/projects/aylien/dynamic-ensembles/data/DUC2004/system_hypotheses/2020-05-14/eval_predicted_summaries.out.5_docs_in_cluster'
# system_hypotheses = '/home/chris/projects/aylien/dynamic-ensembles/data/DUC2004/system_hypotheses/2020-05-14/eval_predicted_summaries.out.8_docs_in_cluster'

# MultiNews
evaluation_dataset = '../data/downloads_from_MultiNews_gdrive/test.jsonl'

# system_hypotheses = '../MultiNews_1_article_eval_predicted_summaries.out'
# system_hypotheses = '../data/downloads_from_MultiNews_gdrive/export_output/transformer.txt'

# system_hypotheses = '../data/downloads_from_MultiNews_gdrive/export_output/hi_map.txt'


# WCEP
evaluation_dataset = '../data/WCEP/test.jsonl'
system_hypotheses = '../wcep_5_articles_eval_predicted_summaries.out'


# TODO: rm tempdir after eval
dataset_rows = [json.loads(l) for l in open(evaluation_dataset)]
orig_system_hyps = [h.strip() for h in open(system_hypotheses)]

In [22]:
# lebanoff_2018_rouge(system_hypotheses, evaluation_dataset)

In [23]:

# Hi-Map Lebanoff Eval
# system_hypotheses = '../data/downloads_from_MultiNews_gdrive/export_output/hi_map.txt'

# ROUGE-1:
# rouge_1_f_score: 0.4028 with confidence interval (0.4002, 0.4056)
# rouge_1_recall: 0.3986 with confidence interval (0.3960, 0.4013)
# rouge_1_precision: 0.4086 with confidence interval (0.4061, 0.4115)

# ROUGE-2:
# rouge_2_f_score: 0.1374 with confidence interval (0.1344, 0.1406)
# rouge_2_recall: 0.1361 with confidence interval (0.1331, 0.1394)
# rouge_2_precision: 0.1392 with confidence interval (0.1360, 0.1424)

# ROUGE-l:
# rouge_l_f_score: 0.3460 with confidence interval (0.3433, 0.3488)
# rouge_l_recall: 0.3424 with confidence interval (0.3398, 0.3452)
# rouge_l_precision: 0.3511 with confidence interval (0.3484, 0.3539)

# ROUGE-s4:
# rouge_s4_f_score: 0.1145 with confidence interval (0.1117, 0.1174)
# rouge_s4_recall: 0.1134 with confidence interval (0.1106, 0.1163)
# rouge_s4_precision: 0.1160 with confidence interval (0.1132, 0.1190)

# ROUGE-su4:
# rouge_su4_f_score: 0.1633 with confidence interval (0.1607, 0.1663)
# rouge_su4_recall: 0.1617 with confidence interval (0.1590, 0.1647)
# rouge_su4_precision: 0.1657 with confidence interval (0.1629, 0.1686)



In [24]:
# dataset_rows = [json.loads(l) for l in open(evaluation_dataset)]
gold_outputs = [json.loads(l)['summary'] for l in open(evaluation_dataset)]
orig_system_hyps = [h.strip() for h in open(system_hypotheses)]

In [25]:
assert len(gold_outputs) == len(orig_system_hyps)
print(f'Evaluation dataset length: {len(gold_outputs)}')

Evaluation dataset length: 1022


In [26]:
# our evaluation implementation (Ghalandari et al 2020)
print_mean(*evaluate_rouge(orig_system_hyps, gold_outputs))

rouge-1 p: 0.297 r: 0.495 f: 0.354
rouge-2 p: 0.125 r: 0.218 f: 0.151
rouge-l p: 0.213 r: 0.364 f: 0.256


In [18]:
# double check multinews evaluation, code taken from: 
# https://github.com/Alex-Fabbri/Multi-News/blob/3675e7c422ae3b4020617a324ac264f50333357d/code/OpenNMT-py-baselines/tools/test_rouge.py
# -*- encoding: utf-8 -*-
import argparse
import os
import time
import pyrouge
import shutil
import sys
import codecs

# from onmt.utils.logging import init_logger, logger


def test_rouge(candidates, references):
    """Calculate ROUGE scores of sequences passed as an iterator
       e.g. a list of str, an open file, StringIO or even sys.stdin
    """
    current_time = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())
    tmp_dir = ".rouge-tmp-{}".format(current_time)
    try:
        if not os.path.isdir(tmp_dir):
            os.mkdir(tmp_dir)
            os.mkdir(tmp_dir + "/candidate")
            os.mkdir(tmp_dir + "/reference")
#         candidates = [line.strip() for line in cand]
#         references = [line.strip() for line in ref]
        assert len(candidates) == len(references)
        cnt = len(candidates)
        for i in range(cnt):
            if len(references[i]) < 1:
                continue
            with open(tmp_dir + "/candidate/cand.{}.txt".format(i), "w",
                      encoding="utf-8") as f:
                f.write(candidates[i])
            with open(tmp_dir + "/reference/ref.{}.txt".format(i), "w",
                      encoding="utf-8") as f:
                f.write(references[i])
        r = pyrouge.Rouge155()
        r.model_dir = tmp_dir + "/reference/"
        r.system_dir = tmp_dir + "/candidate/"
        r.model_filename_pattern = 'ref.#ID#.txt'
        r.system_filename_pattern = 'cand.(\d+).txt'
        rouge_results = r.convert_and_evaluate()
        results_dict = r.output_to_dict(rouge_results)
        return results_dict
    finally:
        pass
        if os.path.isdir(tmp_dir):
            shutil.rmtree(tmp_dir)


def rouge_results_to_str(results_dict):
    return ">> ROUGE(1/2/3/L/SU4): {:.2f}/{:.2f}/{:.2f}/{:.2f}/{:.2f}".format(
        results_dict["rouge_1_f_score"] * 100,
        results_dict["rouge_2_f_score"] * 100,
        results_dict["rouge_3_f_score"] * 100,
        results_dict["rouge_l_f_score"] * 100,
        results_dict["rouge_su*_f_score"] * 100)




In [19]:
fabbri_results = test_rouge(orig_system_hyps, gold_outputs)

In [20]:
fabbri_results

{'rouge_1_recall': 0.40213,
 'rouge_1_recall_cb': 0.39868,
 'rouge_1_recall_ce': 0.40566,
 'rouge_1_precision': 0.54339,
 'rouge_1_precision_cb': 0.54037,
 'rouge_1_precision_ce': 0.54632,
 'rouge_1_f_score': 0.44168,
 'rouge_1_f_score_cb': 0.4389,
 'rouge_1_f_score_ce': 0.44449,
 'rouge_2_recall': 0.14777,
 'rouge_2_recall_cb': 0.14517,
 'rouge_2_recall_ce': 0.15075,
 'rouge_2_precision': 0.19443,
 'rouge_2_precision_cb': 0.19144,
 'rouge_2_precision_ce': 0.19754,
 'rouge_2_f_score': 0.16047,
 'rouge_2_f_score_cb': 0.15786,
 'rouge_2_f_score_ce': 0.1633,
 'rouge_3_recall': 0.08085,
 'rouge_3_recall_cb': 0.07846,
 'rouge_3_recall_ce': 0.0836,
 'rouge_3_precision': 0.10344,
 'rouge_3_precision_cb': 0.10063,
 'rouge_3_precision_ce': 0.10641,
 'rouge_3_f_score': 0.08713,
 'rouge_3_f_score_cb': 0.08471,
 'rouge_3_f_score_ce': 0.0899,
 'rouge_4_recall': 0.05704,
 'rouge_4_recall_cb': 0.05477,
 'rouge_4_recall_ce': 0.05962,
 'rouge_4_precision': 0.07243,
 'rouge_4_precision_cb': 0.06982,
 'r

In [ ]:
# Fabbri results
# system_hypotheses = '../data/downloads_from_MultiNews_gdrive/export_output/hi_map.txt'
{'rouge_1_recall': 0.40213,
 'rouge_1_recall_cb': 0.39868,
 'rouge_1_recall_ce': 0.40566,
 'rouge_1_precision': 0.54339,
 'rouge_1_precision_cb': 0.54037,
 'rouge_1_precision_ce': 0.54632,
 'rouge_1_f_score': 0.44168,
 'rouge_1_f_score_cb': 0.4389,
 'rouge_1_f_score_ce': 0.44449,
 'rouge_2_recall': 0.14777,
 'rouge_2_recall_cb': 0.14517,
 'rouge_2_recall_ce': 0.15075,
 'rouge_2_precision': 0.19443,
 'rouge_2_precision_cb': 0.19144,
 'rouge_2_precision_ce': 0.19754,
 'rouge_2_f_score': 0.16047,
 'rouge_2_f_score_cb': 0.15786,
 'rouge_2_f_score_ce': 0.1633,
 'rouge_3_recall': 0.08085,
 'rouge_3_recall_cb': 0.07846,
 'rouge_3_recall_ce': 0.0836,
 'rouge_3_precision': 0.10344,
 'rouge_3_precision_cb': 0.10063,
 'rouge_3_precision_ce': 0.10641,
 'rouge_3_f_score': 0.08713,
 'rouge_3_f_score_cb': 0.08471,
 'rouge_3_f_score_ce': 0.0899,
 'rouge_4_recall': 0.05704,
 'rouge_4_recall_cb': 0.05477,
 'rouge_4_recall_ce': 0.05962,
 'rouge_4_precision': 0.07243,
 'rouge_4_precision_cb': 0.06982,
 'rouge_4_precision_ce': 0.07518,
 'rouge_4_f_score': 0.06138,
 'rouge_4_f_score_cb': 0.05906,
 'rouge_4_f_score_ce': 0.06398,
 'rouge_l_recall': 0.1952,
 'rouge_l_recall_cb': 0.1929,
 'rouge_l_recall_ce': 0.19779,
 'rouge_l_precision': 0.26637,
 'rouge_l_precision_cb': 0.26372,
 'rouge_l_precision_ce': 0.269,
 'rouge_l_f_score': 0.2138,
 'rouge_l_f_score_cb': 0.21174,
 'rouge_l_f_score_ce': 0.21614,
 'rouge_w_1.2_recall': 0.04501,
 'rouge_w_1.2_recall_cb': 0.04436,
 'rouge_w_1.2_recall_ce': 0.04578,
 'rouge_w_1.2_precision': 0.17602,
 'rouge_w_1.2_precision_cb': 0.17376,
 'rouge_w_1.2_precision_ce': 0.17825,
 'rouge_w_1.2_f_score': 0.06859,
 'rouge_w_1.2_f_score_cb': 0.06773,
 'rouge_w_1.2_f_score_ce': 0.06959,
 'rouge_s*_recall': 0.15868,
 'rouge_s*_recall_cb': 0.15606,
 'rouge_s*_recall_ce': 0.1615,
 'rouge_s*_precision': 0.27614,
 'rouge_s*_precision_cb': 0.27315,
 'rouge_s*_precision_ce': 0.27936,
 'rouge_s*_f_score': 0.17573,
 'rouge_s*_f_score_cb': 0.17353,
 'rouge_s*_f_score_ce': 0.17795,
 'rouge_su*_recall': 0.16112,
 'rouge_su*_recall_cb': 0.15848,
 'rouge_su*_recall_ce': 0.16395,
 'rouge_su*_precision': 0.28006,
 'rouge_su*_precision_cb': 0.27705,
 'rouge_su*_precision_ce': 0.28329,
 'rouge_su*_f_score': 0.17844,
 'rouge_su*_f_score_cb': 0.17623,
 'rouge_su*_f_score_ce': 0.18067}


# Fabbri results
# system_hypotheses = '../MultiNews_1_article_eval_predicted_summaries.out'
{'rouge_1_recall': 0.44188,
 'rouge_1_recall_cb': 0.43897,
 'rouge_1_recall_ce': 0.44475,
 'rouge_1_precision': 0.5006,
 'rouge_1_precision_cb': 0.4972,
 'rouge_1_precision_ce': 0.5039,
 'rouge_1_f_score': 0.45928,
 'rouge_1_f_score_cb': 0.45649,
 'rouge_1_f_score_ce': 0.4619,
 'rouge_2_recall': 0.15733,
 'rouge_2_recall_cb': 0.15442,
 'rouge_2_recall_ce': 0.16038,
 'rouge_2_precision': 0.17746,
 'rouge_2_precision_cb': 0.1743,
 'rouge_2_precision_ce': 0.18095,
 'rouge_2_f_score': 0.1633,
 'rouge_2_f_score_cb': 0.16045,
 'rouge_2_f_score_ce': 0.16641,
 'rouge_3_recall': 0.08404,
 'rouge_3_recall_cb': 0.08145,
 'rouge_3_recall_ce': 0.08676,
 'rouge_3_precision': 0.09447,
 'rouge_3_precision_cb': 0.09159,
 'rouge_3_precision_ce': 0.09757,
 'rouge_3_f_score': 0.08718,
 'rouge_3_f_score_cb': 0.08444,
 'rouge_3_f_score_ce': 0.09007,
 'rouge_4_recall': 0.05816,
 'rouge_4_recall_cb': 0.05576,
 'rouge_4_recall_ce': 0.06072,
 'rouge_4_precision': 0.06546,
 'rouge_4_precision_cb': 0.0628,
 'rouge_4_precision_ce': 0.06827,
 'rouge_4_f_score': 0.06042,
 'rouge_4_f_score_cb': 0.05795,
 'rouge_4_f_score_ce': 0.06308,
 'rouge_l_recall': 0.22003,
 'rouge_l_recall_cb': 0.21755,
 'rouge_l_recall_ce': 0.22262,
 'rouge_l_precision': 0.24691,
 'rouge_l_precision_cb': 0.24421,
 'rouge_l_precision_ce': 0.24985,
 'rouge_l_f_score': 0.22757,
 'rouge_l_f_score_cb': 0.22522,
 'rouge_l_f_score_ce': 0.23011,
 'rouge_w_1.2_recall': 0.04994,
 'rouge_w_1.2_recall_cb': 0.04921,
 'rouge_w_1.2_recall_ce': 0.05076,
 'rouge_w_1.2_precision': 0.16026,
 'rouge_w_1.2_precision_cb': 0.15804,
 'rouge_w_1.2_precision_ce': 0.16262,
 'rouge_w_1.2_f_score': 0.07433,
 'rouge_w_1.2_f_score_cb': 0.0733,
 'rouge_w_1.2_f_score_ce': 0.07548,
 'rouge_s*_recall': 0.18912,
 'rouge_s*_recall_cb': 0.18649,
 'rouge_s*_recall_ce': 0.19164,
 'rouge_s*_precision': 0.24401,
 'rouge_s*_precision_cb': 0.24069,
 'rouge_s*_precision_ce': 0.2473,
 'rouge_s*_f_score': 0.19723,
 'rouge_s*_f_score_cb': 0.19477,
 'rouge_s*_f_score_ce': 0.19965,
 'rouge_su*_recall': 0.19164,
 'rouge_su*_recall_cb': 0.189,
 'rouge_su*_recall_ce': 0.19416,
 'rouge_su*_precision': 0.24676,
 'rouge_su*_precision_cb': 0.24344,
 'rouge_su*_precision_ce': 0.25006,
 'rouge_su*_f_score': 0.19978,
 'rouge_su*_f_score_cb': 0.19732,
 'rouge_su*_f_score_ce': 0.20221}

In [ ]:
if __name__ == "__main__":
#     init_logger('test_rouge.log')
    parser = argparse.ArgumentParser()
    parser.add_argument('-c', type=str, default="candidate.txt",
                        help='candidate file')
    parser.add_argument('-r', type=str, default="reference.txt",
                        help='reference file')
    args = parser.parse_args()
    if args.c.upper() == "STDIN":
        candidates = sys.stdin
    else:
        candidates = codecs.open(args.c, encoding="utf-8")
    references = codecs.open(args.r, encoding="utf-8")

    results_dict = test_rouge(candidates, references)
    logger.info(rouge_results_to_str(results_dict))